In [1]:
import BiomarkerType_Extraction


import cPickle, os, sys, matplotlib

sys.path.insert(1, os.path.join(sys.path[0], '..'))
%load_ext autoreload
%autoreload 2



from snorkel.snorkel import *
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (18,6)
filename = "text/AGR2_blood_biomarker.txt"

In [2]:
Relations = BiomarkerType_Extraction.generateRelations(_filename= filename)
print Relations

Parsing documents...
Parsing contexts...
SENTENCE_______________________


Exception Exception: Exception('Candidate must have word_start and word_end attributes.',) in <generator object apply at 0x109b4fa00> ignored
Exception Exception: Exception('Candidate must have word_start and word_end attributes.',) in <generator object apply at 0x109b4f910> ignored


IndexError: list index out of range

In [ ]:
#NOTE: Relations.pkl is probably wrong file for this learning file
feats = None
relationsFile = "cache/" + filename + "/relations.pkl"
if not os.path.exists("cache/" + filename +"/" ):
    os.makedirs("cache/" + filename + "/")
try:
    with open(relationsFile, 'rb') as f:
        feats = cPickle.load(f)
except:
    %time Relations.extract_features()
    with open(relationsFile, 'w+') as f:
        cPickle.dump(Relations.feats, f)

DDL = DDLiteModel(Relations, feats)
print "Extracted {} features for each of {} mentions".format(DDL.num_feats(), DDL.num_candidates())

In [ ]:
tuples = []
list = re.split("[^\\S ]", open("tags6.tsv").read())
count = 7
while count < len(list):
    number = 0
    if (list[count + 6] == "true"):
        number = 1
    elif (list[count + 6] == "false"):
        number = -1
    tuples.append((list[count + 5] + "::" + list[count + 3] + "::[" + list[count + 4] + ", " + list[count] + "]::['" +
                 list[count + 1] + "', '" + list[count + 2] + "']", number))
    count += 7
gt = []
uids = []
for tuple in tuples:
    uids.append(tuple[0])
    gt.append(tuple[1])


# with open('examples/gene_tag_example/gt/uids.pkl', 'rb') as f:
#     uids2 = cPickle.load(f)
# with open('examples/gene_tag_example/gt/gt.pkl', 'rb') as f:
#     gt2 = cPickle.load(f)
    
DDL.update_gt(gt = gt[:50], uids = uids[:50])
DDL.set_holdout(validation_frac=0.5)

In [ ]:
DDL.open_mindtagger(num_sample=200, width='100%', height=1200)

In [ ]:
DDL.add_mindtagger_tags()
DDL.update_gt(gt=gt[50:], uids=uids[50:])

In [ ]:
negationWords = ["not", "nor", "neither"]

def presenceOfNot(m):
    for word in negationWords:
        if (word in m.post_window1('lemmas', 20)) and (word in m.pre_window2('lemmas', 20)):
            return True
    return False
        
def LF_distance(m):
    distance = abs(m.e2_idxs[0] - m.e1_idxs[0])
    if distance < 8:
        # print "RETURNING ONE"
        return 0
    else:
        return -1

def LF_isAMemberOf(m):
    return 1 if ('is' in m.post_window1('lemmas', 20) and 'a' in m.post_window1('lemmas', 20) and
        'member' in m.post_window1('lemmas', 20) and 'of' in m.post_window1('lemmas', 20)) and (('is' in
        m.pre_window2('lemmas', 20) and 'a' in m.pre_window2('lemmas', 20) and 'member' in
        m.pre_window2('lemmas', 20) and 'of' in m.pre_window2('lemmas', 20))) else 0

def LF_membersOf(m):
    return 1 if ('member' in m.post_window1('lemmas'), 20) and 'of' in m.post_window1('lemmas', 20) and (
    ('member' in m.pre_window2('lemmas'), 20) and 'of' in m.pre_window2('lemmas', 20)) else 0

def LF_family(m):
    return 1 if ('family' in m.post_window1('lemmas', 20) and 'family' in m.post_window2('lemmas', 20)) else 0

def LF_cancerrelated(m):
    return 1 if ('cancer' in m.post_window1('lemmas', 20) and '-' in m.post_window1('lemmas', 20) and
        'related' in m.post_window1('lemmas', 20)) and ('cancer' in m.pre_window2('lemmas', 20) and '-' in m.pre_window1('lemmas', 20) and
        'related' in m.pre_window1('lemmas', 20)) else 0

def LF_isA(m):
    return 1 if ('is' in m.post_window1('lemmas', 20) and 'a' in m.post_window1('lemmas', 20) and
    ('is' in m.pre_window2('lemmas', 20) and 'a' in m.pre_window2('lemmas', 20))) else 0

def LF_types(m):
    return 1 if ('type' in m.post_window1('lemmas', 20) or ('type' in m.pre_window2('lemmas', 20))) else 0

def LF_isaBiomarker(m):
    post_window1_lemmas = m.post_window1('lemmas', 20)
    pre_window2_lemmas = m.pre_window2('lemmas', 20)
    if ('biomarker' in post_window1_lemmas and 'biomarker' in pre_window2_lemmas) or (
            'marker' in post_window1_lemmas and 'marker' in pre_window2_lemmas) or (
            'indicator' in post_window1_lemmas and 'indicator' in pre_window2_lemmas):
        marker_idx_post_window1 = -1
        markers = ['biomarker', 'marker', 'indicator']
        for marker in markers:
            try:
                # print post_window1_lemmas
                findMarker = post_window1_lemmas.index(marker)
                if not findMarker == -1:
                    marker_idx_post_window1 = findMarker
            except:
                pass
        if 'cop' in m.post_window1('dep_labels', 20):
            try:
                cop_idx_post_window1 = m.post_window1('dep_labels', 20).index('cop')
            except:
                pass
            return 1 if ('nsubj' in m.mention1(attribute='dep_labels')) and (
            marker_idx_post_window1 - cop_idx_post_window1 < 4)  else 0
        return 0

In [ ]:
LFs = []
DDL.apply_lfs(LFs, clear=False)

In [ ]:
DDL.lowest_empirical_accuracy_lfs(n=10)

In [ ]:
matplotlib.rcParams['figure.figsize'] = (12,4)
mu_seq = np.ravel([1e-9, 1e-5, 1e-3, 1e-1])
lf_opts = {'sample': False, 'verbose': True}
model_opts = {'sample': False, 'n_iter': 3000, 'alpha': 0.5, 'mu': mu_seq, 'bias': False, 'verbose': True}
%time DDL.train_model(method='lr', lf_opts=lf_opts, model_opts=model_opts)

In [ ]:
DDL.plot_calibration()

In [ ]:
idxs, gt = DDL.get_labeled_ground_truth(subset=DDL.holdout())
acc_feats = np.mean(DDL.get_predicted(subset=DDL.holdout()) == gt)
acc_lfs = np.mean(DDL.get_lf_predicted(subset=DDL.holdout()) == gt)
print "LF accuracy: {:.3f}\nFull model accuracy: {:.3f}".format(acc_lfs, acc_feats)